<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab exercise 1


Per Group:
1.   Create a teams-meeting where one person opens the colab notebook and shares the screen
2.   Change the network as described below
3.   Train the network, then plot the training history and evaluate it one the test set
4.   Post the training history image and the test accuracy in the general Deep Learning teams channel
5.   Think about why your network performed better or worse compared to the ~63% accuracy of the network in the lecture



- **Group A:** Add one more convolutional layer (with stride 1) to the network and train the network
- **Group C:** Add three more convolutional layers (with stride 1) to the network and train the network
- **Group D:** Replace all convolutional layers, the GlobalAveragePooling layer and the first Dense layer with a tf.keras.layers.Flatten() and train the network
- **Group E:** Add an additional Dense layer after the first Dense layer and train the network



## Setup notebook environment

Make sure the Colab environment has a GPU enabled *Edit->Notebook Settings->Hardware accelerator->Choose GPU*

In [ ]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Download the dataset first

In [ ]:
! wget https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
! tar -xf f8d.tar.gz

--2020-10-27 11:28:30--  https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
Resolving pub.ist.ac.at (pub.ist.ac.at)... 81.223.84.195
Connecting to pub.ist.ac.at (pub.ist.ac.at)|81.223.84.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76149807 (73M) [application/x-gzip]
Saving to: ‘f8d.tar.gz’

f8d.tar.gz          100%[===================>]  72.62M  16.6MB/s    in 4.3s    

2020-10-27 11:28:34 (16.8 MB/s) - ‘f8d.tar.gz’ saved [76149807/76149807]



Import python libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

## Step 1: Define datasets

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/test", shuffle=False, batch_size=128, image_size=(224, 224),
)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=64,
  image_size=(224, 224),
  validation_split=0.2,
  subset="training",
  seed=1020202,
)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=128,
  image_size=(224, 224),
  validation_split=0.2,
  subset="validation",
  seed=1020202,
)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 640 files belonging to 8 classes.
Found 1840 files belonging to 8 classes.
Using 1472 files for training.
Found 1840 files belonging to 8 classes.
Using 368 files for validation.


## Step 2 and 3: Define model and objective

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(224,224,3)),
  tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(8, activation="softmax"),
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 110, 110, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 128)       204928    
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 2

## Step 4: Train the model

In [ ]:
training_log = model.fit(
  train_dataset, epochs=150, validation_data=valid_dataset
)

Epoch 1/150
23/23 [==============================] - 56s 2s/step - loss: 2.0704 - sparse_categorical_accuracy: 0.1345 - val_loss: 2.0623 - val_sparse_categorical_accuracy: 0.1386
Epoch 2/150
 5/23 [=====>........................] - ETA: 34s - loss: 2.0408 - sparse_categorical_accuracy: 0.1937

KeyboardInterrupt: ignored

Visualize the training

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(10,10))
axes[0].plot(training_log.history["loss"],label="Training loss")
axes[0].plot(training_log.history["val_loss"],label="Validation loss")
axes[0].legend(loc="upper right")
axes[0].set_ylabel("Loss")
axes[1].plot(training_log.history["sparse_categorical_accuracy"],label="Training accuracy")
axes[1].plot(training_log.history["val_sparse_categorical_accuracy"],label="Validation accuracy")
axes[1].legend(loc="upper right")
axes[1].set_ylabel("Accuracy")
fig.show()

## Step 5: Evaluate the model

In [ ]:
model.evaluate(test_dataset)